In [1]:
import numpy as np
import pandas as pd
from ydata_profiling import ProfileReport

C:\Users\klysm\AppData\Local\Temp\ipykernel_9028\1567141378.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
dataframe = pd.read_csv("SINES.csv") 

In [3]:
df_modelagem = dataframe[['Berth Name', 'Terminal Name', 'Time At Berth', 'Time At Port' , 'Vessel Type - Generic', 'Commercial Market','Voyage Distance Travelled', 'Voyage Speed Average', 'Voyage Origin Port']]

In [4]:
df_modelagem.dropna(how='any', inplace=True)

C:\Users\klysm\AppData\Local\Temp\ipykernel_9028\1220790341.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_modelagem.dropna(how='any', inplace=True)


In [5]:
profile = ProfileReport(df_modelagem, title="Profiling Report filtro colunas")

In [6]:
profile

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
df_modelagem.describe()

,Time At Berth,Time At Port,Voyage Distance Travelled,Voyage Speed Average
count,3912.000000,3912.000000,3912.000000,3912.000000
mean,8.583257,23.681442,282.053937,9.068865
std,20.685901,47.219557,1062.650691,4.000634
min,0.000000,0.200000,1.000000,0.000000
25%,0.400000,3.000000,3.000000,5.800000
50%,0.800000,12.400000,3.000000,7.600000
75%,8.900000,27.300000,6.000000,12.000000
max,673.700000,888.600000,12658.000000,36.900000


In [8]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

df_modelagem = df_modelagem.copy()

# Encode the categorical variables
categorical_cols = ['Berth Name', 'Terminal Name', 'Vessel Type - Generic', 'Commercial Market', 'Voyage Origin Port']
for col in categorical_cols:
    df_modelagem[col] = LabelEncoder().fit_transform(df_modelagem[col])

In [18]:
df_modelagem.describe()

,Berth Name,Terminal Name,Time At Berth,Time At Port,Vessel Type - Generic,Commercial Market,Voyage Distance Travelled,Voyage Speed Average,Voyage Origin Port
count,3912.000000,3912.000000,3912.000000,3912.000000,3912.000000,3912.000000,3912.000000,3912.000000,3912.000000
mean,7.972648,1.226994,8.583257,23.681442,16.839468,6.592791,282.053937,9.068865,109.892127
std,3.138012,1.108119,20.685901,47.219557,7.876722,2.538092,1062.650691,4.000634,27.530660
min,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,1.000000,0.000000,0.000000
25%,5.000000,0.000000,0.400000,3.000000,6.000000,5.000000,3.000000,5.800000,120.000000
50%,8.000000,2.000000,0.800000,12.400000,20.000000,8.000000,3.000000,7.600000,120.000000
75%,11.000000,2.000000,8.900000,27.300000,24.000000,8.000000,6.000000,12.000000,120.000000
max,12.000000,4.000000,673.700000,888.600000,25.000000,9.000000,12658.000000,36.900000,137.000000


In [9]:
from sklearn.model_selection import train_test_split

X = df_modelagem.drop('Time At Berth', axis=1)
y = df_modelagem['Time At Berth']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
from xgboost import XGBRegressor

model = XGBRegressor(objective='reg:squarederror', random_state=42)
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [11]:
y_pred = model.predict(X_test)

In [12]:
from sklearn.metrics import mean_squared_error, r2_score

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse:.2f}')
print(f'R-squared: {r2:.2f}')

Mean Squared Error: 416.96
R-squared: 0.56


In [13]:
from sklearn.neural_network import MLPRegressor

# Create the MLP model
mlp_model = MLPRegressor(hidden_layer_sizes=(50, 50), activation='relu', solver='adam', random_state=22, max_iter=1000)

# Train the MLP model
mlp_model.fit(X_train, y_train)

MLPRegressor(hidden_layer_sizes=(50, 50), max_iter=1000, random_state=22)

In [14]:
y_pred_mlp = mlp_model.predict(X_test)

# Calculate model performance
mse_mlp = mean_squared_error(y_test, y_pred_mlp)
r2_mlp = r2_score(y_test, y_pred_mlp)

print(f'MLP Mean Squared Error: {mse_mlp:.2f}')
print(f'MLP R-squared: {r2_mlp:.2f}')

MLP Mean Squared Error: 683.70
MLP R-squared: 0.28


In [16]:
comparison_df = pd.DataFrame({
    'True Value': y_test,
    'Predicted Value': y_pred
})

# Display the first few rows of the DataFrame
print(comparison_df.head(10))

      True Value  Predicted Value
3000         0.3         0.403625
2006         1.1         1.845453
2480         0.5         0.601008
1467         7.7         7.565701
2754         3.1         3.564274
2884        29.4        28.882505
218         27.3        25.829014
2393         1.4         1.350883
2701         6.4         5.555290
1482         0.4         1.062494
